<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/Models/Final%20Models/BiLSTM-CRF/SENNA/Average%20BiLSTM-CRF%20(SENNA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite
!pip install keras==2.2.4
%tensorflow_version 1.x
#!pip install tensorflow-gpu==1.15.0

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-urst8dq3
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-urst8dq3
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=45271faab11c76c6ed4864af205cad8f89f534d3f8d45684108af5b8f7dcf683
  Stored in directory: /tmp/pip-ephem-wheel-cache-p0cf5ptw/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import re
import numpy as np
import urllib.request

#keras and tensorflow packages
import tensorflow as tf
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras_contrib.metrics import crf_accuracy

#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score, flat_recall_score
from sklearn.metrics import confusion_matrix
print(tf.__version__)

Using TensorFlow backend.


1.15.2


In [ ]:
#Ensure Reproducibility
import numpy as np
import random as rn
import os

random_seed=3

# 1) set numpy random seed
np.random.seed(random_seed)
# 2) set build-in random seed
rn.seed(random_seed)
# 3) set PYTHONHASHSEED
os.environ['PYTHONHASHSEED']=str(random_seed)
# 4) set tensorflow random seed
tf.random.set_random_seed(random_seed)

# 5) Force TensorFlow to use single thread.
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

def readstring(filename, meth):
    f = filename.split('\n')
    sentences = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        if meth.lower()=="numbers1":
            sentence.append([hasNumbers1(splits[0]), splits[-1].strip()])
        elif meth.lower()=="numbers2":
            sentence.append([hasNumbers2(splits[0]), splits[-1].strip()])
        else:
            sentence.append([splits[0], splits[-1].strip()])
    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
    return sentences

def hasNumbers1(inputString):
    if re.search(r'\d', inputString):
        return "0"
    else:return(inputString)

def hasNumbers2(text):
  if text.isdigit():
      return "1"
  elif re.search(r'\d',text) and re.search(r'\,|\.',text):
      return "1" 
  else:
      if re.search(r'\d', text):
          return(re.sub('\d','D', text))
      else:
          return text

#preproces the txt file
train_data = readstring(train,"Numbers1")
test_data = readstring(test,"Numbers1")

#create corpus
corpus = train_data.copy()
corpus.extend(test_data)

In [ ]:
def reformat_data(data,meth):
  if meth.lower() == "data":
    i=0
  else: i=1
  train = []
  output= []
  for sentence in data:
    words=[]
    for x in sentence:
      words.append(x[i])
    train.append(words)

  for i in train:
    string = ' '.join(i)
    output.append(string)
  return output

def get_max_length(corpus):
  length = []
  for sentence in corpus:
    length.append(len(sentence))
  return int(max(length))

def number_of_tags(corpus):
  tags=[]
  for sentence in corpus:
    for tag in sentence:
      tags.append(tag[1])
  return int(len(list(set(tags))))


MAX_LEN = get_max_length(corpus)
N_tags = number_of_tags(corpus)

train = reformat_data(train_data,"data")
test = reformat_data(test_data,"data")


In [ ]:
# create a tokenizer
token_word = text.Tokenizer(char_level=False, lower=True, filters="}", oov_token='UNK')
token_word.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
X_train = sequence.pad_sequences(token_word.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
X_test = sequence.pad_sequences(token_word.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")

In [ ]:
train = reformat_data(train_data,"tags")
test = reformat_data(test_data,"tags")

# create a tokenizer
token_tag = text.Tokenizer(char_level=False, lower=False, filters="}")
token_tag.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
Y = sequence.pad_sequences(token_tag.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_train = [to_categorical(i, num_classes=N_tags+1) for i in Y]

Y = sequence.pad_sequences(token_tag.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_test = [to_categorical(i, num_classes=N_tags+1) for i in Y]

#add padding 
token_tag.index_word[0]="PAD"
sub_label = list(token_tag.index_word.values())
sub_label.remove('O')
sub_label.remove('PAD')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = "/content/drive/My Drive/SENNA.txt"
embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 130000 word vectors.


In [ ]:
#create emedding matrix
EMBEDDING=50
word_index = token_word.word_index
embedding_matrix = np.zeros((len(token_word.word_index) + 1, EMBEDDING))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
      embedding_matrix[i] = embedding_vector

In [ ]:
checks=[]
words=[]
for i in range(0,len(token_word.word_index)+1):
  if embedding_matrix[i][0] == 0.0:
    checks.append(1)
    words.append(list(token_word.word_index.items())[i-1][0])
  else:
    continue

print("Missing words from Embeddings: %d (%.2f%%)" %(len(checks),(len(checks)/len(token_word.word_index)*100)))

Missing words from Embeddings: 1553 (9.05%)


## Create Model

In [ ]:
%%time
from keras import optimizers
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Paramters
EPOCHS=20
BATCH_SIZE = 64
LSTM_CELLS = 100
DROPOUT = 0.25
HIDDEN = 50
CLIP = 2.0

#optimizer
OPT=optimizers.Adam(clipvalue=CLIP)

#number of runs
runs = 3
results = []

#Build Model
def create_model():
  input =  Input(shape=(MAX_LEN,))

  model_2 = Embedding(len(token_word.word_index) + 1,output_dim=EMBEDDING,
                    weights=[embedding_matrix],input_length=MAX_LEN,
                    trainable=False,mask_zero=True)(input)

  model_2 = Bidirectional(LSTM(units=LSTM_CELLS, return_sequences=True,
                            recurrent_dropout=DROPOUT, dropout=DROPOUT))(model_2)

  model_2 = TimeDistributed(Dense(HIDDEN, activation="relu"))(model_2)
  crf = CRF(N_tags+1)  # CRF layer
  out = crf(model_2)  # output
  model_2 = Model(input, out)
  model_2.compile(optimizer=OPT, loss=crf_loss,metrics=[crf_viterbi_accuracy])
  return model_2

for i in range(1,runs+1):
  print("Run:",(i))
  model = create_model()
  history = model.fit(X_train, np.array(Y_train), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1,callbacks=[early_stopping])
  
  print("Predicting")
  pred = model.predict(X_test, verbose=1)

  preds = np.argmax(pred, axis=-1)
  y_tr_true = np.argmax(Y_test, -1)
  # Convert the index to tag
  pred_tag = [[token_tag.index_word[i] for i in row] for row in preds]
  y_tr_true_tag = [[token_tag.index_word[i] for i in row] for row in y_tr_true]

  #basic metrics
  f1_score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)
  recall_score = flat_recall_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)
  pre_score = flat_precision_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)

  #create report
  report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag, labels=sub_label, output_dict=True)
  df = pd.DataFrame(report).transpose()

  #create confusion matrix
  flat_preds = [item for sublist in pred_tag for item in sublist]
  flat_true = [item for sublist in y_tr_true_tag for item in sublist]
  sub_label.append("O")
  matrix=confusion_matrix(flat_preds,flat_true, labels=sub_label)
  cm = pd.DataFrame(matrix, columns = sub_label, index=sub_label)
  sub_label.remove("O")



  results.append({"Precision":pre_score,
                  "Recall": recall_score,
                  "F1 Score": f1_score,
                  "report": df,
                  "matrix": cm,
                  "Run": i})

Run: 1







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 11232 samples, validate on 2809 samples
Epoch 1/20
11232/11232 [==============================] - 180s 16ms/step - loss: 40.4006 - crf_viterbi_accuracy: 0.8817 - val_loss: 35.2603 - val_crf_viterbi_accuracy: 0.9383
Epoch 2/20
11232/11232 [==============================] - 180s 16ms/step - loss: 40.1227 - crf_viterbi_accuracy: 0.9436 - val_loss: 35.2170 - val_crf_viterbi_accuracy: 0.9472
Epoch 3/20
11232/11232 [==============================] - 180s 16ms/step - loss: 40.0866 - crf_viterbi_accuracy: 0.9513 - val_loss: 35.1974 - val_crf_viterbi_accuracy: 0.9508
Epoch 4/20
11232/11232 [==============================] - 180s 16ms/step - loss: 40.0653 - crf_viterbi_accuracy: 0.9573 - val_loss: 35.1838 - val_crf_viterbi_accuracy: 0.9553
Epoch 5/20
11232/11232 [=

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run: 2
Train on 11232 samples, validate on 2809 samples
Epoch 1/20
11232/11232 [==============================] - 186s 17ms/step - loss: 40.2010 - crf_viterbi_accuracy: 0.9220 - val_loss: 35.2094 - val_crf_viterbi_accuracy: 0.9469
Epoch 2/20
11232/11232 [==============================] - 183s 16ms/step - loss: 40.0665 - crf_viterbi_accuracy: 0.9566 - val_loss: 35.1813 - val_crf_viterbi_accuracy: 0.9551
Epoch 3/20
11232/11232 [==============================] - 183s 16ms/step - loss: 40.0418 - crf_viterbi_accuracy: 0.9632 - val_loss: 35.1679 - val_crf_viterbi_accuracy: 0.9601
Epoch 4/20
11232/11232 [==============================] - 182s 16ms/step - loss: 40.0281 - crf_viterbi_accuracy: 0.9674 - val_loss: 35.1621 - val_crf_viterbi_accuracy: 0.9618
Epoch 5/20
11232/11232 [==============================] - 182s 16ms/step - loss: 40.0208 - crf_viterbi_accuracy: 0.9695 - val_loss: 35.1543 - val_crf_viterbi_accuracy: 0.9639
Epoch 6/20
11232/11232 [==============================] - 181s 16ms/s

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run: 3
Train on 11232 samples, validate on 2809 samples
Epoch 1/20
11232/11232 [==============================] - 185s 16ms/step - loss: 40.2032 - crf_viterbi_accuracy: 0.9226 - val_loss: 35.2016 - val_crf_viterbi_accuracy: 0.9514
Epoch 2/20
11232/11232 [==============================] - 183s 16ms/step - loss: 40.0627 - crf_viterbi_accuracy: 0.9577 - val_loss: 35.1782 - val_crf_viterbi_accuracy: 0.9569
Epoch 3/20
11232/11232 [==============================] - 184s 16ms/step - loss: 40.0378 - crf_viterbi_accuracy: 0.9644 - val_loss: 35.1650 - val_crf_viterbi_accuracy: 0.9613
Epoch 4/20
11232/11232 [==============================] - 183s 16ms/step - loss: 40.0261 - crf_viterbi_accuracy: 0.9671 - val_loss: 35.1598 - val_crf_viterbi_accuracy: 0.9634
Epoch 5/20
11232/11232 [==============================] - 185s 16ms/step - loss: 40.0171 - crf_viterbi_accuracy: 0.9704 - val_loss: 35.1543 - val_crf_viterbi_accuracy: 0.9648
Epoch 6/20
11232/11232 [==============================] - 184s 16ms/s

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: user 2h 52min 13s, sys: 13min 51s, total: 3h 6min 5s
Wall time: 2h 10min 29s


In [ ]:
from google.colab import files
#df_results = pd.DataFrame(results)

count = 1
for i in results:
  report = i["report"]
  matrix = i["matrix"]

  report.to_csv('Senna_report_'+str(count)+'.csv')
  files.download('Senna_report_'+str(count)+'.csv')

  matrix.to_csv('Senna_matrix_'+str(count)+'.csv')
  files.download('Senna_matrix_'+str(count)+'.csv')
  count +=1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_results = pd.DataFrame(results)
df_results.drop(['report',"matrix"], axis=1, inplace=True)

df_results.to_csv('Average_BiLSTM_Senna.csv')
files.download('Average_BiLSTM_Senna.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#mean Results
df_results.mean()

Precision    0.836140
Recall       0.820348
F1 Score     0.828124
Run          2.000000
dtype: float64

In [ ]:
df1 = results[0]["report"]
for i in range(1,3):
  df1 = df1 + results[i]["report"]

avg_report = df1/3

In [ ]:
avg_report.sort_values(by=['support'], ascending=False)

,precision,recall,f1-score,support
micro avg,0.836140,0.820348,0.828124,8112.0
macro avg,0.464590,0.456540,0.459963,8112.0
weighted avg,0.832936,0.820348,0.825358,8112.0
I-PER,0.877462,0.941099,0.908016,2773.0
I-ORG,0.806118,0.725411,0.763526,2491.0
I-LOC,0.880907,0.839500,0.859525,1919.0
I-MISC,0.687645,0.689769,0.688676,909.0
B-MISC,0.000000,0.000000,0.000000,9.0
B-LOC,0.000000,0.000000,0.000000,6.0
B-ORG,0.000000,0.000000,0.000000,5.0
